# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     |  Jesus Alejandro Garcia Garcia |
| **Fecha**      |  11-09-25 |
| **Expediente** |  751749 |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso

In [3]:
df=pd.read_csv("Advertising.csv")
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [4]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
df.describe()

,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.describe()

,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


## Sin penalizacion

In [12]:
cols=df.columns
cols=cols.drop("sales")
x_list = []

for col in cols:
    x_list.append(df[col].values.reshape(-1, 1))


X = np.hstack(x_list)
y = df["sales"]

In [13]:
lr=LinearRegression()
lr.fit(X,y)

print("Intercept:", lr.intercept_)
print("Coefficients:", lr.coef_)
print("R^2:", r2_score(y, lr.predict(X)))

Intercept: 2.938889369459403
Coefficients: [ 0.04576465  0.18853002 -0.00103749]
R^2: 0.8972106381789521


In [ ]:
y_media=np.mean(y)
TSS=np.sum((y - y_media)**2)
RSS=(1-r2_score(y, lr.predict(X)))*(TSS)
print(f'El RSS es: {RSS}')
x=np.ones((len(df),1))
X_o=np.hstack((x,X))
ols=sm.OLS(y,X_o)
results=ols.fit()
results.summary()

El RSS es: 556.8252629021875


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        11:43:43   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Penalizacion en L2

In [60]:
ridge = Ridge(alpha=1)
ridge.fit(X, y)

print("Ridge Intercept:", ridge.intercept_)
print("Ridge Coefficients:", ridge.coef_)
print("Ridge R^2:", r2_score(y, ridge.predict(X)))


Ridge Intercept: 2.938967458330149
Ridge Coefficients: [ 0.04576464  0.1885251  -0.00103629]
Ridge R^2: 0.8972106380074802


In [65]:
n=len(y)
p=4
x_media=np.mean(X, axis=0)
TSS=np.sum((y - y_media)**2)
RSS_ridge=sum((ridge.predict(X)-y)**2)
sigma2=(RSS_ridge/(n-p))
SE_Bo=np.sqrt(sigma2* (1/n + (x_media**2)/np.sum((X - x_media)**2)))
SE_B1=np.sqrt(sigma2/sum((X - x_media)**2))
t_Bo=ridge.intercept_/SE_Bo[0]
t_B1=ridge.coef_[0]/SE_B1[0]
t_B2=ridge.coef_[1]/SE_B1[1]
t_B3=ridge.coef_[2]/SE_B1[2]
pv_bo=2*(1-stats.t.cdf(np.abs(t_Bo),n-p))
pv_b1=2*(1-stats.t.cdf(np.abs(t_B1),n-p))
pv_b2=2*(1-stats.t.cdf(np.abs(t_B2),n-p))
pv_b3=2*(1-stats.t.cdf(np.abs(t_B3),n-p))
p_v_ridge=[pv_bo,pv_b1,pv_b2,pv_b3]

print(f'Los valores p de ridge son: {p_v_ridge}')


Los valores p de ridge son: [0.0, 0.0, 0.0, 0.8503746527570268]


Cuanto más aumentaba el alpha del ridge, menores se hacían los p-values, lo que significa que aumentaba la significancia de las x y su aporte al modelo.

## Penalizacion en L1

In [66]:
lasso=Lasso(alpha=1)
lasso.fit(X,y)
print("Lasso Intercept:", lasso.intercept_)
print("Lasso Coefficients:", lasso.coef_)
print("Lasso R^2:", r2_score(y, lasso.predict(X)))

Lasso Intercept: 3.0402155834803786
Lasso Coefficients: [0.04566142 0.1834644  0.        ]
Lasso R^2: 0.8970235728389689


In [67]:
RSS_lasso=sum((lasso.predict(X)-y)**2)
sigma2_lasso=(RSS_lasso/(n-p))
SE_Bo_lasso=np.sqrt(sigma2_lasso* (1/n + (x_media**2)/np.sum((X - x_media)**2)))
SE_B1_lasso=np.sqrt(sigma2_lasso/sum((X - x_media)**2))
t_Bo_lasso=lasso.intercept_/SE_Bo_lasso[0]
t_B1_lasso=lasso.coef_[0]/SE_B1_lasso[0]
t_B2_lasso=lasso.coef_[1]/SE_B1_lasso[1]
t_B3_lasso=lasso.coef_[2]/SE_B1_lasso[2]
pv_bo=2*(1-stats.t.cdf(np.abs(t_Bo_lasso),n-p))
pv_b1=2*(1-stats.t.cdf(np.abs(t_B1_lasso),n-p))
pv_b2=2*(1-stats.t.cdf(np.abs(t_B2_lasso),n-p))
pv_b3=2*(1-stats.t.cdf(np.abs(t_B3_lasso),n-p))
p_v_lasso=[pv_bo,pv_b1,pv_b2,pv_b3]

print(f'Los valores p de lasso son: {p_v_lasso}')

Los valores p de lasso son: [0.0, 0.0, 0.0, 1.0]


Cuanto más aumentaba su alpha, mayores valores tenían los p-values de los coeficientes.